# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [2]:
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
from azureml.train.sklearn import SKLearn
from azureml.widgets import RunDetails

from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.dataset import Dataset
from azureml.core import Environment
from azureml.core import Workspace
from azureml.core import Experiment

import joblib

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
ws = Workspace.from_config()
experiment_name = 'hyperdrive-exp-iris'
project_folder = './outputs'
experiment=Experiment(ws, experiment_name)

In [4]:
# get and register data
example_data = 'https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data'
dataset = Dataset.Tabular.from_delimited_files(example_data, header=False)        
#Register Dataset in Workspace
dataset = dataset.register(workspace=ws,
                                   name="Iris Classification",
                                   description="Classification problem for iris species")


df = dataset.to_pandas_dataframe()
df.head()

,Column1,Column2,Column3,Column4,Column5
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [6]:
# setup compute
hyperdrive_compute_cluster = 'com1'

compute_target = ComputeTarget(workspace=ws, name=hyperdrive_compute_cluster)
compute_target.wait_for_completion(show_output=True)


Running


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [7]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(
    evaluation_interval=2, slack_factor=0.1
)

if "training" not in os.listdir():
    os.mkdir("./training")

#TODO: Create the different params that you will be using during training
ps = RandomParameterSampling({
        "--C": uniform(0.1, 1),
        "--max_iter": choice(50, 75, 100, 150)
})

#TODO: Create your estimator and hyperdrive config
est = SKLearn(source_directory='./', 
    compute_target=compute_target, 
    entry_script='train.py',
    vm_size = "Standard_D2_V2")

hyperdrive_config = HyperDriveConfig(
            estimator=est,
            policy=early_termination_policy,
            hyperparameter_sampling=ps,
            primary_metric_name='Accuracy',
            primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
            max_total_runs=10,
            max_concurrent_runs=5,
)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.
'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


In [8]:
#TODO: Submit your experiment
hdr_exp = experiment.submit(hyperdrive_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [9]:
RunDetails(hdr_exp).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [13]:
hdr_exp.wait_for_completion()

{'runId': 'HD_845b306e-6d37-4f5f-9c9f-1f610cf347c7',
 'target': 'com1',
 'status': 'Completed',
 'startTimeUtc': '2021-07-30T23:21:33.791011Z',
 'endTimeUtc': '2021-07-30T23:25:09.660946Z',
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': 'b035a180-7f31-45c3-ac95-91a789be5639',
  'user_agent': 'python/3.6.9 (Linux-5.4.0-1051-azure-x86_64-with-debian-buster-sid) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.31.0',
  'score': '0.9777777777777777',
  'best_child_run_id': 'HD_845b306e-6d37-4f5f-9c9f-1f610cf347c7_0',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://mlstrg153057.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_845b306e-6d37-4f5f-9c9f-1f610cf347c7/azureml-logs/hyperdrive.txt?sv=2019-02-02&sr=b&sig=jdPV

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [14]:
# best model metrics and details
best_run_hdr = hdr_exp.get_best_run_by_primary_metric()
best_run_hdr.get_details()
print(hdr_exp.get_children_sorted_by_primary_metric(top=1))

[{'run_id': 'HD_845b306e-6d37-4f5f-9c9f-1f610cf347c7_8', 'hyperparameters': '{"--C": 0.7772069615440829, "--max_iter": 100}', 'best_primary_metric': 0.9777777777777777, 'status': 'Completed'}]


In [16]:
#TODO: Save the best model
# joblib.dump(best_run_hdr, './output/hdr_best_run_model.joblib')
registered_model = best_run_hdr.register_model(model_name="best_hdr_iris", model_path="outputs/model.joblib")
registered_model.download(target_dir="./output", exist_ok=True)

'output/model.joblib'

In [ ]:
# compute_target.delete()

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
# Deployed automl model which has higher accuracy. 

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
# Deployed automl model which has higher accuracy.

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
# Deployed automl model which has higher accuracy.